In [1]:
import ee; ee.Initialize()

In [2]:
import ee.mapclient


In [4]:
%matplotlib inline
from IPython.display import Image


In [11]:
# Import the Folium library.
import folium

# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, eeImageObject, visParams, name):
    mapID = ee.Image(eeImageObject).getMapId(visParams)
    folium.raster_layers.TileLayer(
    tiles = "https://earthengine.googleapis.com/map/"+mapID['mapid']+
      "/{z}/{x}/{y}?token="+mapID['token'],
    attr = "Map Data &copy; <a href='https://earthengine.google.com/'>Google Earth Engine</a>",
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

In [12]:
OldIMAGE  = ee.Image( 'CGIAR/SRTM90_V4' )
MaskIMAGE = ee.Image( 'MCD12Q1/MCD12Q1_005_2001_01_01').select(['Land_Cover_Type_1']).neq(9)
MaskedIMAGE  = OldIMAGE.mask( MaskIMAGE );

In [13]:
# Create a folium map object.
myMap = folium.Map(location=[20,0], zoom_start=3, height=500)

# Set visualization parameters.
visParams = {'min':0, 'max':3000, 'palette':['225ea8','41b6c4','a1dab4','ffffcc']}
visParamsold = { 'min':0, 'max':1400, 'palette':['000000','ff00ff']}
visParamsmask = { 'min':0, 'max':1400, 'palette':['000000','0000ff']}

# Add to the map object.
myMap.add_ee_layer(OldIMAGE, visParamsold, 'Old')
myMap.add_ee_layer(MaskedIMAGE, visParamsmask, 'Masked')

# Add a layer control panel to the map.
myMap.add_child(folium.LayerControl())

# Display the map.
display(myMap)

In [5]:
image = ee.Image('srtm90_v4')
print(image.getInfo())


{'type': 'Image', 'bands': [{'id': 'elevation', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -32768, 'max': 32767}, 'dimensions': [432000, 144000], 'crs': 'EPSG:4326', 'crs_transform': [0.000833333333333, 0.0, -180.0, 0.0, -0.000833333333333, 60.0]}], 'version': 1494271934303000, 'id': 'srtm90_v4', 'properties': {'system:time_start': 950227200000, 'system:time_end': 951177600000, 'system:asset_size': 18827626666}}


In [7]:
image1 = ee.Image('srtm90_v4')
path = image1.getDownloadUrl({
    'scale': 30,
    'crs': 'EPSG:4326',
    'region': '[[-120, 35], [-119, 35], [-119, 34], [-120, 34]]'
})
print(path)

https://earthengine.googleapis.com/api/download?docid=57067cb8d7f7618d88b4fa052546f82c&token=83f32f0c73781b21aa5245bb246fcd74


In [15]:
probav = ee.Image('LC8_L1T_TOA/LC80410362015107LGN00')

# Reduce image collection to mean
probav_mean = probav.mean()

# Set location to Europe (Proba-V Footprint: X18Y02) 
point = ee.Geometry.Point(6.134136, 49.612485)
luxembourg = point.buffer(50000).bounds().getInfo()['coordinates']


AttributeError: 'Image' object has no attribute 'mean'

In [9]:
area_of_interest = ee.Geometry.Rectangle([-98.75, 19.15, -98.15,18.75])
mexico_landcover_2010_landsat = ee.Image("users/renekope/MEX_LC_2010_Landsat_v43").clip(area_of_interest)
landsat8_collection = ee.ImageCollection('LANDSAT/LC8_L1T_TOA').filterDate('2016-01-01', '2018-04-19').min()
landsat8_collection = landsat8_collection.slice(0,9)


In [10]:
vis = {
    'bands': ['B6', 'B5', 'B2'],
    'min': 0,
    'max': 0.5,
    'gamma': [0.95, 1.1, 1],
    'region':area_of_interest.getInfo()['coordinates']} 
image = landsat8_collection.clip(area_of_interest)
theGeom = area_of_interest.getInfo()['coordinates']
thumbnail = image.getThumbUrl(vis)
Image(url=thumbnail)


In [23]:
ee.Image('LC8_L1T_TOA/LC80410362015107LGN00')
Map.setCenter( -118.2733, 34.0942, 12 ); 


In [17]:
ndvi = probav_mean.normalizedDifference(["NIR", "RED"])
Image(url=ndvi.getThumbUrl({
    'region':luxembourg,
    'min':-0.5,
    'max':0.5,
    'palette':['ffffe5','f7fcb9','d9f0a3','addd8e','78c679','41ab5d','238443','006837','004529']
}))


In [21]:
Image(url=probav_mean.getThumbUrl({
    'region':luxembourg,
    'min':0,
    'max':255,
    'bands':['NDVI'],
    'palette':['ffffe5','f7fcb9','d9f0a3','addd8e','78c679','41ab5d','238443','006837','004529']
}))


In [16]:
def NDVI(image):
    return image.normalizedDifference(['B5', 'B4'])

def SAM(image):
    band1 = image.select("B1")
    bandn = image.select("B2","B3","B4","B5","B6","B7","B8","B9");
    maxObjSize = 256;
    b = band1.divide(bandn);
    spectralAngleMap = b.atan();
    spectralAngleMap_sin = spectralAngleMap.sin();
    spectralAngleMap_cos = spectralAngleMap.cos();
    sum_cos = spectralAngleMap_cos.reduce(ee.call("Reducer.sum"));
    sum_sin = spectralAngleMap_sin.reduce(ee.call("Reducer.sum"));
    return ee.Image.cat(sum_sin, sum_cos, spectralAngleMap_sin, spectralAngleMap_cos);

#Enhanced Vegetation Index
def EVI(image):
    # L(Canopy background)
    # C1,C2(Coefficients of aerosol resistance term)
    # GainFactor(Gain or scaling factor)
    gain_factor = ee.Image(2.5);
    coefficient_1 = ee.Image(6);
    coefficient_2 = ee.Image(7.5);
    l = ee.Image(1);
    nir = image.select("B5");
    red = image.select("B4");
    blue = image.select("B2");
    evi = image.expression(
        "Gain_Factor*((NIR-RED)/(NIR+C1*RED-C2*BLUE+L))",
        {
            "Gain_Factor":gain_factor,
            "NIR":nir,
            "RED":red,
            "C1":coefficient_1,
            "C2":coefficient_2,
            "BLUE":blue,
            "L":l
        }
    )
    return evi

#Atmospherically Resistant Vegetation Index
def ARVI(image):
    red = image.select("B4")
    blue = image.select("B2")
    nir = image.select("B5")
    red_square = red.multiply(red)
    arvi = image.expression(
        "NIR - (REDsq - BLUE)/(NIR+(REDsq-BLUE))",{
            "NIR": nir,
            "REDsq": red_square,
            "BLUE": blue
        }
    )
    return arvi

#Leaf Area Index
def LAI(image):
    nir = image.select("B5")
    red = image.select("B4")
    coeff1 = ee.Image(0.0305);
    coeff2 = ee.Image(1.2640);
    lai = image.expression(
        "(((NIR/RED)*COEFF1)+COEFF2)",
        {
            "NIR":nir,
            "RED":red,
            "COEFF1":coeff1,
            "COEFF2":coeff2
        }
    )
    return lai

def tasseled_cap_transformation(image):
    #Tasseled Cap Transformation for Landsat 8 based on the 
    #scientfic work "Derivation of a tasselled cap transformation based on Landsat 8 at-satellite reflectance" 
    #by M.Baigab, L.Zhang, T.Shuai & Q.Tong (2014). The bands of the output image are the brightness index, 
    #greenness index and wetness index.
    b = image.select("B2", "B3", "B4", "B5", "B6", "B7");
    #Coefficients are only for Landsat 8 TOA
    brightness_coefficents= ee.Image([0.3029, 0.2786, 0.4733, 0.5599, 0.508, 0.1872])
    greenness_coefficents= ee.Image([-0.2941, -0.243, -0.5424, 0.7276, 0.0713, -0.1608]);
    wetness_coefficents= ee.Image([0.1511, 0.1973, 0.3283, 0.3407, -0.7117, -0.4559]);
    fourth_coefficents= ee.Image([-0.8239, 0.0849, 0.4396, -0.058, 0.2013, -0.2773]);
    fifth_coefficents= ee.Image([-0.3294, 0.0557, 0.1056, 0.1855, -0.4349, 0.8085]);
    sixth_coefficents= ee.Image([0.1079, -0.9023, 0.4119, 0.0575, -0.0259, 0.0252]);
    
    #Calculate tasseled cap transformation
    brightness = image.expression(
        '(B * BRIGHTNESS)',
        {
            'B':b,
            'BRIGHTNESS': brightness_coefficents
        })
    greenness = image.expression(
        '(B * GREENNESS)',
        {
            'B':b,
            'GREENNESS': greenness_coefficents
        })
    wetness = image.expression(
        '(B * WETNESS)',
        {
            'B':b,
            'WETNESS': wetness_coefficents
        })
    fourth = image.expression(
        '(B * FOURTH)',
        {
            'B':b,
            'FOURTH': fourth_coefficents
        })
    fifth = image.expression(
        '(B * FIFTH)',
        {
            'B':b,
            'FIFTH': fifth_coefficents
        })
    sixth = image.expression(
        '(B * SIXTH)',
        {
            'B':b,
            'SIXTH': sixth_coefficents
        })
    bright = brightness.reduce(ee.call("Reducer.sum"));
    green = greenness.reduce(ee.call("Reducer.sum"));
    wet = wetness.reduce(ee.call("Reducer.sum"));
    four = fourth.reduce(ee.call("Reducer.sum"));
    five = fifth.reduce(ee.call("Reducer.sum"));
    six = sixth.reduce(ee.call("Reducer.sum"));
    tasseled_cap = ee.Image(bright).addBands(green).addBands(wet).addBands(four).addBands(five).addBands(six)
    return tasseled_cap.rename('brightness','greenness','wetness','fourth','fifth','sixth')

In [17]:
tct = tasseled_cap_transformation(landsat8_collection)


In [18]:
image = tct.clip(area_of_interest)
theGeom = area_of_interest.getInfo()['coordinates']
thumbnail = image.getThumbUrl({'min':-1,'max':2,'size':'800','bands':['brightness','greenness','wetness'],
                               'region':theGeom})
Image(url=thumbnail)


In [19]:
ndvi = NDVI(landsat8_collection)
sam = SAM(landsat8_collection)
evi = EVI(landsat8_collection)
arvi = ARVI(landsat8_collection)
lai = LAI(landsat8_collection)
spectral_indices_stack = ee.Image(ndvi).addBands(lai).addBands(sam).addBands(arvi).addBands(evi).addBands(tct).addBands(landsat8_collection)


In [20]:
image = ndvi.clip(area_of_interest)
theGeom = area_of_interest.getInfo()['coordinates']
thumbnail = image.getThumbUrl({'min':-1,'max':1,'size':'800',
                               'region':theGeom})
Image(url=thumbnail)


In [21]:
def classification(raster_input, training_dataset,number_of_training_points, region, classification_algorithm):
    bands = raster_input.bandNames()
    points = ee.FeatureCollection.randomPoints(region, number_of_training_points, number_of_training_points, 1)
    training = training_dataset.addBands(raster_input).reduceToVectors(
        reducer='mean',
        geometry=points,
        geometryType='centroid',
        scale=30,
        crs='EPSG:4326'
    )
    classifier = ee.Classifier.randomForest().train(
        features=training,
        classProperty='label',
        inputProperties=raster_input.bandNames(),
    )
    out = raster_input.classify(classifier)
    return out

In [22]:
output = classification(spectral_indices_stack, mexico_landcover_2010_landsat, 10000, area_of_interest, 'Cart')


In [23]:
palette = ['5d9cd4','007e00','003c00','aaaa00','aa8000','8baa00','ffb265','00d900','aa007f','ff55ff','ff557f','ff007f','ff55ff','aaffff','00ffff','55aaff','e29700','bd7e00','966400','a2ecb1','c46200','aa5500','6d3600','00aa7f','008a65','005941','e9e9af','faff98',
'00007f','c7c8bc','4d1009','000000','fef7ff','6daa50','3a7500','0b5923','ffaaff','ffd1fa']
palette = ','.join(palette)

# make a visualizing variable
vis = {'min': 0, 'max': len(palette), 'palette': palette, 'region':theGeom}


In [24]:
image = mexico_landcover_2010_landsat.clip(area_of_interest)
theGeom = area_of_interest.getInfo()['coordinates']
thumbnail = image.getThumbUrl(vis)
Image(url=thumbnail)


In [25]:
image = output.clip(area_of_interest)
theGeom = area_of_interest.getInfo()['coordinates']
thumbnail = image.getThumbUrl(vis)
Image(url=thumbnail)
